In [1]:
import os
import h5py

In [2]:
import librosa

In [3]:
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
import numpy as np
import pandas as pd

In [6]:
from scipy.stats import kurtosis
from scipy.stats import skew

In [7]:
from collections import defaultdict
import pickle

In [8]:
import ast

In [9]:
#I took help from github link: https://github.com/celestinhermez/music-genre-classification/blob/master/data_preprocessing/create_audio_features.ipynb
# path to the small directory
SMALL_AUDIO_DIR = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small'

# function to get the paths to all the songs in the small dataset
def audio_paths(AUDIO_DIR):
    AUDIO_PATHS = []
    # iterate through all the directories with songs in them
    for path in [os.path.join('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small', p) 
                 for p in os.listdir('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small') 
                 if not (p.endswith('checksums') or p.endswith('.txt') or p.endswith('.DS_Store'))]:
        # add all songs to the list
        AUDIO_PATHS = AUDIO_PATHS + [os.path.join(path, track).replace('\\', '/') for track in os.listdir(path)]
    
    return AUDIO_PATHS

# store all the small paths
SMALL_PATHS = audio_paths(SMALL_AUDIO_DIR)

In [10]:
print(SMALL_PATHS[0:15])

['E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000002.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000005.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000010.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000140.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000141.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000148.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000182.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000190.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000193.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000194.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000197.mp3', 'E:/8th Semester/CSE4836-Patter

In [11]:
t=pd.read_csv('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv')
t.head(3)

C:\Users\HP\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food


In [12]:
# function to load metadata
# adapted from https://github.com/mdeff/fma/blob/master/utils.py
def metadata_load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    pd.CategoricalDtype(categories=SUBSETS, ordered=True))
        except ValueError:
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [13]:
# function to get genre information for each track ID
def track_genre_information(GENRE_PATH, TRACKS_PATH, FILE_PATHS, subset):
    """
    GENRE_PATH (str): path to the csv with the genre metadata
    TRACKS_PATH (str): path to the csv with the track metadata
    FILE_PATHS (list): list of paths to the mp3 files
    subset (str): the subset of the data desired
    """
    # get the genre information
    genres = pd.read_csv(GENRE_PATH)

    # load metadata on all the tracks
    tracks = metadata_load(TRACKS_PATH)
    print(tracks.head(5))

    # focus on the specific subset tracks
    subset_tracks = tracks[tracks['set', 'subset'] <= subset]

    # extract track ID and genre information for each track
    subset_tracks_genre = np.array([np.array(subset_tracks.index), 
                                  np.array(subset_tracks['track', 'genre_top'])]).T
    
    # extract track indices from the file paths
    track_indices = []
    for path in FILE_PATHS:
        track_indices.append(path.split('/')[-1].split('.')[0].lstrip('0'))

    # get the genre associated with each file path, thanks to the path ID
    print(track_indices)
    track_indices = pd.DataFrame({'file_path':FILE_PATHS,'track_id':np.array(track_indices).astype(int)})
    tracks_genre_df = pd.DataFrame({'track_id': subset_tracks_genre[:,0], 'genre': subset_tracks_genre[:,1]})
    track_genre_data = track_indices.merge(tracks_genre_df, how='left')
    
    # label classes with numbers
    encoder = LabelEncoder()
    track_genre_data['genre_nb'] = encoder.fit_transform(track_genre_data.genre)
    
    return track_genre_data

# get genre information for all tracks from the small subset
GENRE_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/genres.csv'
TRACKS_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv'
subset = 'small'

small_tracks_genre = track_genre_information(GENRE_PATH, TRACKS_PATH, SMALL_PATHS, subset)

            album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []  

In [14]:
small_tracks_genre.head(4)

,file_path,track_id,genre,genre_nb
0,E:/8th Semester/CSE4836-Pattern Recognition La...,2,Hip-Hop,3
1,E:/8th Semester/CSE4836-Pattern Recognition La...,5,Hip-Hop,3
2,E:/8th Semester/CSE4836-Pattern Recognition La...,10,Pop,6
3,E:/8th Semester/CSE4836-Pattern Recognition La...,140,Folk,2


In [15]:
print(small_tracks_genre.shape)

(7997, 4)


In [16]:
small_tracks_genre = small_tracks_genre.drop([120,121,122,123,124,125], axis=0)

In [17]:
print(small_tracks_genre.shape)

(7991, 4)


In [18]:
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew

In [19]:
splitted_sample_size = 0.1
overlap_with_prev = 0.5

In [20]:
n_fft=1024
hop_length=256

In [21]:
path1 = small_tracks_genre.iloc[7]['file_path']
path2 = small_tracks_genre.iloc[765]['file_path']
signal, rate = librosa.load(path1)
print(signal.shape)
signal, rate = librosa.load(path2)
print(signal.shape)

C:\Users\HP\Anaconda\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(661560,)


C:\Users\HP\Anaconda\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(660984,)


In [22]:
#as song returned from librosa is of different shape, I have consider till 660000 for the ease of generalization
consider_output = 660000

In [23]:
def split_track(X, y):
    splitted_X = []
    splitted_y = []
    no_of_values = X.shape[0]
    values_in_each_split = int(no_of_values*splitted_sample_size)
    offset = int(values_in_each_split*(1.-overlap_with_prev))
    end_at = no_of_values - values_in_each_split + offset
    
    splitted_samples = [X[i:i+values_in_each_split] for i in range(0, end_at , offset)]
    for i in splitted_samples:
        #this condition is to avoid those samples at the end
        if i.shape[0] == values_in_each_split: 
            splitted_X.append(i)
            splitted_y.append(y)
    return np.array(splitted_X), np.array(splitted_y)

In [24]:
def get_melspectrogram(song, n_fft=1024, hop_length=256):
    mel_values_of_splits = []
    for i in song:
        temp_val = librosa.feature.melspectrogram(i, n_fft=n_fft,hop_length=hop_length, n_mels=128)[:,:,np.newaxis]
        mel_values_of_splits.extend(temp_val)
    return np.array(list(mel_values_of_splits))

In [25]:
def extract_features(X, y):
    splitted_features = []
    splitted_genres = []
    f = 0
    for song, genre in zip(X, y):
        print("File=",song)
        print(f)
        f = f+1
        signal, rate = librosa.load(song)
        signal = signal[:consider_output]

        # Splitsongs into 0.15 of each to take sample of 1.5 seconds only
        signals, y = split_track(signal, genre)
        # Get Labrosa Melspectogram
        splits = get_melspectrogram(signals)
        splitted_features.extend(y)
        splitted_genres.extend(splits)
    return splitted_features, splitted_genres

In [26]:
#Creating a Dictionary of Genres
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

In [27]:
small_tracks_genre = small_tracks_genre.sample(frac=1, random_state=66)

In [28]:
small_tracks_genre=small_tracks_genre.groupby('genre', group_keys=False).apply(lambda x: x.sample(125))

In [29]:
print(small_tracks_genre.value_counts(subset='genre'))

genre
Rock             125
Pop              125
International    125
Instrumental     125
Hip-Hop          125
Folk             125
Experimental     125
Electronic       125
dtype: int64


In [30]:
print(small_tracks_genre.shape)

(1000, 4)


In [31]:
list_songs = []
list_genres = []

for ind in small_tracks_genre.index:
    song_link = small_tracks_genre['file_path'][ind]
    song_genre = small_tracks_genre['genre'][ind]
    list_songs.append(song_link)
    list_genres.append(genres[song_genre])

X_train, X_test, y_train, y_test = train_test_split(list_songs, list_genres, test_size=0.15, random_state=47, stratify=list_genres)

In [32]:
print(len(X_train) , len(y_train), len(X_test), len(y_test))

850 850 150 150


In [33]:
sample_x_train = len(X_train)
sample_x_test = len(X_test)

In [34]:
X_return, y_return = extract_features(X_train, y_train)

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113303.mp3
0


C:\Users\HP\Anaconda\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036965.mp3
1
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/005/005158.mp3
2
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055900.mp3
3
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137899.mp3
4
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108027.mp3
5
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071243.mp3
6
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032693.mp3
7
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012060.mp3
8
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145259.mp3
9
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061589.mp3
10
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/040/040525.mp3
89
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047526.mp3
90
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042016.mp3
91
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148210.mp3
92
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081543.mp3
93
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148289.mp3
94
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126718.mp3
95
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/011/011020.mp3
96
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141590.mp3
97
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133560.mp3
98
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Da

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123934.mp3
176
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110384.mp3
177
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/044/044795.mp3
178
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113789.mp3
179
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115474.mp3
180
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105914.mp3
181
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131933.mp3
182
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032021.mp3
183
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125621.mp3
184
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087967.mp3
185
File= E:/8th Semester/CSE4836-Pattern Recognition 

...

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145610.mp3
437
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092947.mp3
438
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115321.mp3
439
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113264.mp3
440
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/058/058333.mp3
441
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/138/138211.mp3
442
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129920.mp3
443
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132122.mp3
444
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073467.mp3
445
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143303.mp3
446
File= E:/8th Semester/CSE4836-Pattern Recognition 

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123986.mp3
524
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069205.mp3
525
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/028/028482.mp3
526
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116344.mp3
527
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133803.mp3
528
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111149.mp3
529
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145258.mp3
530
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127280.mp3
531
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042048.mp3
532
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121660.mp3
533
File= E:/8th Semester/CSE4836-Pattern Recognition 

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/082/082250.mp3
611
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068541.mp3
612
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126944.mp3
613
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110438.mp3
614
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118087.mp3
615
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145703.mp3
616
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010250.mp3
617
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006358.mp3
618
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/149/149625.mp3
619
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/030/030519.mp3
620
File= E:/8th Semester/CSE4836-Pattern Recognition 

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/062/062188.mp3
698
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054623.mp3
699
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120325.mp3
700
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000708.mp3
701
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118478.mp3
702
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055807.mp3
703
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036799.mp3
704
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/037/037417.mp3
705
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033460.mp3
706
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107020.mp3
707
File= E:/8th Semester/CSE4836-Pattern Recognition 

File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036428.mp3
785
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117170.mp3
786
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/049/049812.mp3
787
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143046.mp3
788
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113336.mp3
789
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/084/084141.mp3
790
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121474.mp3
791
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/040/040542.mp3
792
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063471.mp3
793
File= E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012692.mp3
794
File= E:/8th Semester/CSE4836-Pattern Recognition 

In [35]:
w = len(X_return)
print(w)

16150


In [36]:
sample_per_music_file = int(w/sample_x_train)
print(sample_per_music_file)

19


In [37]:
temp_arr = np.array(X_return)
print(temp_arr.shape)

(16150,)


In [39]:
print(temp_arr[19])

1


In [38]:
temp_arr = np.array(X_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX_new', temp_arr)

In [39]:
temp_arr = np.array(y_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay_new', temp_arr)

In [40]:
X_return, y_return = split_then_mels(X_test, y_test)

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117670.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074389.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127359.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134943.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145606.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148606.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120203.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089843.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116733.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/005/005

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123503.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055783.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092889.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116451.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/144/144487.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010389.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056467.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/005/005521.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026011.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

In [41]:
w = len(X_return)
print(w)

5850


In [42]:
temp_arr = np.array(X_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatestX_new', temp_arr)

In [43]:
temp_arr = np.array(y_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatesty_new', temp_arr)

In [44]:
del temp_arr
del X_return
del y_return

# CNN Model

In [1]:
import os
import h5py
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import tensorflow as tf
from tensorflow import keras

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.random.seed(467)

In [3]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX_new.npy'
X_train = np.load(fileXtrain)

In [4]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay_new.npy'
y_train = np.load(fileXtrain)

In [5]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatestX_new.npy'
X_test = np.load(fileXtrain)

In [6]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatesty_new.npy'
y_test = np.load(fileXtrain)

In [7]:
print(X_train.shape, X_train.shape, y_train.shape, y_test.shape)

(33150, 128, 129, 1) (33150, 128, 129, 1) (33150,) (5850,)


In [8]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
print(X_train.shape, X_train.shape, y_train.shape, y_test.shape)

(33150, 128, 129, 1) (33150, 128, 129, 1) (33150, 8) (5850, 8)


In [10]:
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

# Data Augmentation

In [11]:
from tensorflow.keras.utils import Sequence
class FMAGenerator(Sequence):
    def __init__(self, X, y, batch_size=64, is_test = False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.is_test = is_test
    
    def __len__(self):
        return int(np.ceil(len(self.X)/self.batch_size))
    
    def __getitem__(self, index):
        # Get batch indexes
        signals = self.X[index*self.batch_size:(index+1)*self.batch_size]

        # Apply data augmentation
        if not self.is_test:
            signals = self.__augment(signals)
        return signals, self.y[index*self.batch_size:(index+1)*self.batch_size]
    
    def __augment(self, signals, hor_flip = 0.5, random_cutout = 0.5):
        spectrograms =  []
        for s in signals:
            signal = copy(s)
            
            # Perform horizontal flip
            if np.random.rand() < hor_flip:
                signal = np.flip(signal, 1)

            # Perform random cutoout of some frequency/time
            if np.random.rand() < random_cutout:
                lines = np.random.randint(signal.shape[0], size=3)
                cols = np.random.randint(signal.shape[0], size=4)
                signal[lines, :, :] = -80 # dB
                signal[:, cols, :] = -80 # dB

            spectrograms.append(signal)
        return np.array(spectrograms)
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        np.random.shuffle(self.indexes)
        return None

# Defining The Network

In [12]:
def conv_block(x, n_filters, pool_size=(2, 2)):
    x = Conv2D(n_filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=pool_size, strides=pool_size)(x)
    x = Dropout(0.25)(x)
    return x

In [13]:
# Model Definition
def create_model(input_shape, num_genres):
    inpt = Input(shape=input_shape)
    x = conv_block(inpt, 16)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    
    # Global Pooling and MLP
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    x = Dropout(0.25)(x)
    predictions = Dense(num_genres, 
                        activation='softmax', 
                        kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    
    model = Model(inputs=inpt, outputs=predictions)
    return model

In [14]:
model = create_model(X_train[0].shape, 8)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 129, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 129, 16)      160       
_________________________________________________________________
activation (Activation)      (None, 128, 129, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0     

In [16]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [17]:
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.95,
    patience=3,
    verbose=1,
    mode='min',
    min_delta=0.0001,
    cooldown=2,
    min_lr=1e-5
)

In [26]:
# Generators
batch_size = 64
train_generator = FMAGenerator(X_train, y_train)
steps_per_epoch = np.ceil(len(X_train)/batch_size)

validation_generator = FMAGenerator(X_test, y_test)
val_steps = np.ceil(len(X_test)/batch_size)

In [27]:
import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [28]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)

config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Num GPUs Available:  1


In [29]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=val_steps,
    epochs=250,
    verbose=1,
    callbacks=[reduceLROnPlat])

Epoch 1/250


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d/Conv2D (defined at <ipython-input-20-398df8729895>:1) ]] [Op:__inference_train_function_1346]

Function call stack:
train_function


In [ ]:
model.save("cnn_final3.h5")

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("val_loss = {:.3f} and val_acc = {:.3f}".format(score[0], score[1]))

In [ ]:
new_model = tf.keras.models.load_model('cnn_final3.h5')
new_model.summary()

# Max-Voting Approach

In [ ]:
sample_per_music_file = 7

In [ ]:
y_pred = new_model.predict(X_test)

In [ ]:
print(y_pred.shape[0])
print(y_pred.shape[1])

In [ ]:
def evaluate_performance(y_pred,y_test):
    total_files = int(y_test.shape[0] / sample_per_music_file)
    no_of_right_pred = 0
    no_of_wrong_pred = 0
    for i in range(0, total_files):
        start_at = i * sample_per_music_file 
        end_at = start_at + sample_per_music_file
        count_in = np.zeros(y_pred.shape[1])
        for j in range(start_at, end_at):
            label_now = np.argmax(y_pred[j])
            count_in[label_now] = count_in[label_now] + 1
        pred_label = np.argmax(count_in)
        true_label = np.argmax(y_test[start_at])
        
        if pred_label == true_label:
            no_of_right_pred = no_of_right_pred + 1
        else:
            no_of_wrong_pred = no_of_wrong_pred + 1
            
    print('Right:',no_of_right_pred)
    print('Wrong:',no_of_wrong_pred)
    
    accuracy = no_of_right_pred / total_files
    
    return accuracy
            
            
        

In [ ]:
y_pred = new_model.predict(X_test)
calculate_accuracy = evaluate_performance(y_pred,y_test)
print("Test_acc = {:.3f}".format(calculate_accuracy))